In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/metagem

/content/gdrive/MyDrive/metagem


In [ ]:
from Utils.utils import Utils

In [ ]:
from AEME import AEME

In [ ]:
if __name__ == "__main__":
  utils = Utils()
  tens_1 = utils.get_embeddingtensor('/content/gdrive/MyDrive/metagem/Data/FB15k/HOPE_emb.csv')
  tens_2 = utils.get_embeddingtensor('/content/gdrive/MyDrive/metagem/Data/FB15k/LLE_emb.csv')


In [ ]:
tens_1 = tens_1.float()
tens_2 = tens_2.float()

In [ ]:
vocab = utils.read_data('/content/gdrive/MyDrive/metagem/Data/FB15k/subgraph_nodes.edgelist')
len(vocab)
vocab
for i in range(len(vocab)):
  vocab[i] = vocab[i].strip()

In [ ]:
emb = {}
model = ["CAE"]
for md in model:
  ae = AEME(mode = md)
  if md == "DAE":
    model_checkpoint = "/content/gdrive/MyDrive/metagem/Results/CAE/_epoch_500_loss_0.39228.pt"
    tensor_dataset = ae.prepare_input(vocab, tens_1, tens_2)
    emb[md] = ae.predict(tensor_dataset, model_checkpoint)
  else:
    model_checkpoint = "/content/gdrive/MyDrive/metagem/Results/" + md +"/_epoch_500_loss_0.01200.pt"
    tensor_dataset = ae.prepare_input(vocab, tens_1, tens_2)
    emb[md] = ae.predict(tensor_dataset, model_checkpoint)

CPU on


In [ ]:
l = []
with open ("/content/gdrive/MyDrive/metagem/Data/FB15k/train.txt") as o:
  for line in o:
        l.append(line)
triple = utils.split_raw_triple(l)
#triple

In [ ]:
triple = utils.delete_edge(triple)
for edge in triple:
    edge[1] = edge[1].strip()

In [ ]:
key = utils.nodemap(triple)
keyinv = utils.mapback(key)
#keyinv

In [ ]:
triple = utils.preprocess(triple, key)
#triple = utils.drop_duplicate_node(triple)
#triple

In [ ]:
sgnodesn = utils.drop_duplicate_node(utils.getsubgraph(triple, [0, 2, 32, 540]))
sgkey = utils.nodemap(sgnodesn)
sgkeyinv = utils.mapback(sgkey)
#sgkeyinv

In [ ]:
no = []
with open("/content/gdrive/MyDrive/metagem/Data/FB15k/subgraph_nodes.edgelist") as op:
  for line in op:
    no.append(line)
no

for n in range(len(no)):
  no[n] = int(no[n].strip())

In [ ]:
len(no)

1256

In [ ]:
node_name = {}
k = list(sgkey.keys())
for el in keyinv:
  for e in k:
    if e == el:
      node_name[sgkeyinv[sgkey[el]]] = keyinv[el]

#node_name

In [ ]:
len(emb["CAE"]['736'])

200

In [ ]:
item = list(emb["CAE"].items())
l = len(item)

In [ ]:
import numpy as np
X = np.zeros((l, 200))
for i in range(len(X)):
  X[i : ] = item[i][1]
  


In [ ]:
X.shape

(1256, 200)

In [ ]:
import networkx as nx
G = utils.loadGraphFromEdgeListTxt("/content/gdrive/MyDrive/metagem/Data/FB15k/subgraphnew.edgelist")
G = G.to_directed()

from Utils import graphreconstruct as gr
node_num = len(G.nodes)
estimated_adj = gr.get_reconstructed_adj(X, node_num)

In [ ]:
estimated_adj

array([[0.        , 1.        , 1.        , ..., 1.        , 0.99999999,
        1.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 0.99999999,
        1.        ],
       [1.        , 1.        , 0.        , ..., 1.        , 0.99999999,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 0.99999999,
        1.        ],
       [0.99999999, 0.99999999, 0.99999999, ..., 0.99999999, 0.        ,
        0.99999999],
       [1.        , 1.        , 1.        , ..., 1.        , 0.99999999,
        0.        ]])

In [ ]:
estimated_edgelist = gr.get_edge_list_from_adj_mtrx(estimated_adj, threshold=1)

In [ ]:
from Utils import metrics
MAP = metrics.computeMAP(estimated_edgelist, G, is_undirected=False)
prec_curv, _ = metrics.computePrecisionCurve(estimated_edgelist, G)
print(("\tMAP: {} \t preccision curve: {}\n\n\n\n"+'-'*100).format(MAP, prec_curv[:5]))

	MAP: 0.5679829102516755 	 preccision curve: [0.0, 0.0, 0.0, 0.0, 0.0]



----------------------------------------------------------------------------------------------------


In [ ]:
len(estimated_edgelist)

550240

In [ ]:
print(estimated_edgelist[0])

(0, 1, 1.0)
